<a href="https://colab.research.google.com/github/Batmaev/qua/blob/main/Permutations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
try:
    import cirq
except ImportError:
    !pip install --quiet cirq --pre
    import cirq
import sympy
import scipy.optimize
import numpy as np

# Задача коммивояжёра

Ответом на задачу является последовательность, в которой мы посещаем города.

Первый город можно выбрать $n$ способами. Если граф полный, то второй город можно выбрать $n-1$ способом, третий $n-2$ способами и так далее, суммарно $n!$ возможных последовательностей городов.

Будем искать незамкнутые пути.

In [ ]:
# n_towns = 4

In [ ]:
def create_cost_matrix(n_towns):
   """Рандомная симметричная матрица с положительными элементами, 
   которая хранит длины рёбер между каждой парой городов"""

   A = np.random.random((n_towns, n_towns))
   A -= np.tril(A)
   
   return A.T + A

In [ ]:
# cost_matrix_4 = create_cost_matrix(4)

cost_matrix_4 = np.array([[0.        , 0.04711181, 0.01197604, 0.85992487],
                          [0.04711181, 0.        , 0.68829915, 0.80606249],
                          [0.01197604, 0.68829915, 0.        , 0.46924138],
                          [0.85992487, 0.80606249, 0.46924138, 0.        ]])

In [ ]:
def cost_of_permutation(cost_matrix, town_sequence):
  cost = 0.0

  for i in range(len(town_sequence)-1):
    cost += cost_matrix [town_sequence[i]] [town_sequence[i+1]]
  
  return cost

#### Аналитическое решение

Переберём все перестановки в лексикографическом порядке.

Будем итерироваться по номеру перестановки. Сначала переведём его в факториальную систему счисления [(factoradic)](https://en.wikipedia.org/wiki/Factorial_number_system). Полученные цифры будут совпадать с [кодом Лемера](https://en.wikipedia.org/wiki/Lehmer_code) перестановки. 

Затем по коду Лемера вычислим саму перестановку и затем её стоимость.

In [ ]:
# Для перевода из десятичной системы счисления в факториальную 
# понадобятся факториалы, поэтому вычислим их заранее:
#
# factorials[n] == n!

factorials = [1]
for i in range(30):
  # Предполагаю, что n_towns < 30
  factorials.append(factorials[i] * (i+1))

In [ ]:
def factoradic_from_perm_number(perm_number : int, factoradic_array):
  """Преобразует число в факториальную систему счисления.
  Записывает полученные цифры в factoradic_array.
  Нулевая цифра в массиве - самая значимая. Последняя цифра всегда равна 0.
  Алгоритм из https://en.wikipedia.org/wiki/Factorial_number_system#Definition """

  N = perm_number
  n_towns = len(factoradic_array)

  for i in range(len(factoradic_array)):
    factoradic_array[i] = N // factorials[n_towns - i - 1]
    N %= factorials[n_towns - i - 1]
  
  return factoradic_array

In [ ]:
def permutation_from_factoradic(nums : np.ndarray):
  """Перезаписывает массив nums.
  Если изначально там находилось число в факториальной системе, то потом там будет находиться перестановка.
  Города в полученной перестановке нумеруются с 0. 
  Алгоритм из https://en.wikipedia.org/wiki/Lehmer_code#Encoding_and_decoding """
  for i in range(len(nums)-2, -1, -1):
    for j in range(i+1, len(nums)):
      if(nums[j] >= nums[i]):
        nums[j] += 1
  return nums

In [ ]:
# Тест из Википедии
permutation_from_factoradic([1, 4, 0, 3, 1, 1, 0]) \
                         == [1, 5, 0, 6, 3, 4, 2]

True

In [ ]:
def analytical(cost_matrix):

  n_towns = len(cost_matrix)

  factoradic_array = np.ndarray(n_towns, int)

  min_cost = np.inf
  opt_perm_number = np.nan

  for perm_number in range(factorials[n_towns]):
    cost = cost_of_permutation(
        cost_matrix,
        permutation_from_factoradic(
            factoradic_from_perm_number(perm_number, factoradic_array)
            )
        )
    if cost < min_cost:
      min_cost = cost
      opt_perm_number = perm_number

  return {"min_cost": min_cost, 
          "order_of_towns": permutation_from_factoradic(
              factoradic_from_perm_number(opt_perm_number, factoradic_array)
            )}

In [ ]:
analytical(cost_matrix_4)

{'min_cost': 0.52832923, 'order_of_towns': array([1, 0, 2, 3])}

## Цепь

In [ ]:
class FactoradicCirquit:
   """
Схема из нескольких пар слоёв Rx и CNOT.
Может подобрать оптимальные углы для гейтов Rx.

3-я цифра ├ 0: ───Rx(a_0)───@───────────────────Rx(a_6)────@───────────────────
                            │                              │
    2-ая  ┌ 1: ───Rx(a_1)───X───@───────────────Rx(a_7)────X───@───────────────
    цифра |                     │                              │
          └ 2: ───Rx(a_2)───────X───@───────────Rx(a_8)────────X───@───────────
                                    │                              │
          ┌ 3: ───Rx(a_3)───────────X───@───────Rx(a_9)────────────X───@───────
    1-ая  |                             │                              │
    цифра | 4: ───Rx(a_4)───────────────X───@───Rx(a_10)───────────────X───@───
          |                                 │                              │
          └ 5: ───Rx(a_5)───────────────────X───Rx(a_11)───────────────────X───

Блоки из кубитов кодируют цифры в факториальной системе счисления.
Закодированная цифра равна числу кубитов в блоке таких, что |<1|ψ>| > 0.5
   """

   def __init__(self, n_towns, n_layer_pairs):
      # n_qubits = 1 + 2 + ... + n-1 = (n-1)n/2
      n_qubits = (n_towns - 1) * n_towns // 2

      self.qubits = cirq.LineQubit.range(n_qubits)
      self.circuit = cirq.Circuit()
      self.observables = [(cirq.Z(qubit)+1)/2 for qubit in self.qubits]
      self.simulator = cirq.Simulator()

      self._factoradic_array = np.ndarray(n_towns, int)

      self._angle_names = [f'a_{i}' for i in range(n_layer_pairs * n_qubits)]
      angle_symbols = sympy.symbols(self._angle_names)


      for layer_pair in range(n_layer_pairs):
          rx_gates = [cirq.rx(angle)(qubit)
                      for (angle, qubit)
                      in zip(angle_symbols[layer_pair*n_qubits : (layer_pair+1)*n_qubits],    self.qubits) ]
          self.circuit.append(cirq.Moment(rx_gates))

          cnot_gates = [cirq.CNOT(q1, q2)
                        for (q1, q2)
                        in zip(self.qubits, self.qubits[1:]) ]
          self.circuit.append(cnot_gates)


   def __str__(self):
      return self.circuit.__str__()

   def _repr_pretty_(self, *args):
      "Text output in Jupyter"
      return self.circuit._repr_pretty_(*args)


   def calc_observables(self, angles, initial_state=None):
      """
      Аргумент 'angles' должен быть массивом длины n_qubits * n_layer_pairs,
      где n_qubits = n_towns * (n_towns - 1) / 2.
      """

      params = cirq.ParamResolver({
         name: value for (name, value) in
         zip(self._angle_names, angles)
      })

      return self.simulator.simulate_expectation_values_sweep(
         self.circuit, self.observables, params, initial_state=initial_state
      )


   def calc_factoradic(self, angles, initial_state = None):
      """
      Запускает симуляцию с заданными углами для гейтов Rx
      и возвращает номер полученной перестановки в факториальной системе счисления.
      Этот номер в виде массива цифр также записывается в self._factoradic_array.
      """

      [bits] = np.real(self.calc_observables(angles, initial_state))
      # Значения observbles лежат от 0 до 1. Округлим их, чтобы получить нули и единицы.
      bits = np.rint(bits)


      bit_pos = 0
      for digit_number_from_end in range(len(self._factoradic_array)):

        # Нулевая цифра с конца равна нулю.
        # Первая цифра с конца меньше либо равна 1, вторая - меньше либо равна 2 и так далее.
        #
        # Для каждой цифры суммируем digit_number_from_end элементов массива bits.

        digit_value = 0
        for _ in range(digit_number_from_end):
          digit_value += bits[bit_pos]
          bit_pos += 1 

        self._factoradic_array[- 1 - digit_number_from_end] = digit_value

      return self._factoradic_array


   def cost(self, angles, cost_matrix, initial_state=None):
      self.calc_factoradic(angles, initial_state)
      return cost_of_permutation(
          cost_matrix,
          permutation_from_factoradic(self._factoradic_array)
      )


   def optimize_without_constraints(self, cost_matrix, optimizer = scipy.optimize.basinhopping,
                                    initial_state = None, **optimizer_kwargs):
      """
      Вызывает функцию, заданную в аргументе optimizer.
      С её помощью подбирает параметры гейтов так,
      чтобы минимизировать суммарную длину пути в задаче коммивояжёра.
      Маршрут вычисляется на основе округлённых до 0 и 1 средних значений наблюдаемых,
      заданных в self.observables.

      Args:

       cost_matrix - матрица размером n_towns × n_towns такая, что
                     cost_matrix[i][j] есть длина пути между городами i и j

       optimizer - одна из функций https://docs.scipy.org/doc/scipy/reference/optimize.html#global-optimization

      """

      cost_lambda  = lambda angles : self.cost(angles, cost_matrix, initial_state)

      if optimizer == scipy.optimize.basinhopping:
         if 'x0' in optimizer_kwargs:
            return optimizer(cost_lambda, **optimizer_kwargs)
         else:
            x0 = [2 * np.pi * (np.random.random() - 0.5)
                            for _ in self._angle_names]
            return optimizer(cost_lambda, x0, **optimizer_kwargs)
      else:
         if 'bounds' in optimizer_kwargs:
            return optimizer(cost_lambda, **optimizer_kwargs)
         else:
            bounds = [(-np.pi, np.pi) for _ in self._angle_names]
            return optimizer(cost_lambda, bounds, **optimizer_kwargs)

In [ ]:
qaoaInstance_4_1 = FactoradicCirquit(n_towns = 4, n_layer_pairs = 1)
qaoaInstance_4_1

0: ───Rx(a_0)───@───────────────────
                │
1: ───Rx(a_1)───X───@───────────────
                    │
2: ───Rx(a_2)───────X───@───────────
                        │
3: ───Rx(a_3)───────────X───@───────
                            │
4: ───Rx(a_4)───────────────X───@───
                                │
5: ───Rx(a_5)───────────────────X───

В качестве классического оптимизатора я пробовал использовать:

| Алгоритм | Комментарий | Максимальная протестированная цепь |
|---------:|-------------|---------------|
|`scipy.optimize.basinhopping`| быстро работает, но не всегда точно, может понадобиться запустить несколько раз| 6 городов, 2 пары слоёв (2 min) |
| `dual_annealing` | хорошо работает, но медленно | 6 городов, 1 пара слоёв (14 min) |
|`differential_evolution` | хорошо работает, но медленно | 6 городов, 1 пара слоёв (22 min) |
| `shgo` | плохо работает |

По умолчанию используется `basinhopping`.

In [ ]:
# with scipy.optimize.basinhopping
optimizeResult_4_1 = qaoaInstance_4_1.optimize_without_constraints(cost_matrix_4)
optimizeResult_4_1

                        fun: 0.52832923
 lowest_optimization_result:       fun: 0.52832923
 hess_inv: array([[1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 1]])
      jac: array([0., 0., 0., 0., 0., 0.])
  message: 'Optimization terminated successfully.'
     nfev: 7
      nit: 0
     njev: 1
   status: 0
  success: True
        x: array([2.06846712, 1.56650307, 0.34412043, 1.15650966, 1.04056943,
       2.4276422 ])
                    message: ['requested number of basinhopping iterations completed successfully']
      minimization_failures: 0
                       nfev: 777
                        nit: 100
                       njev: 111
                          x: array([2.06846712, 1.56650307, 0.34412043, 1.15650966, 1.04056943,
       2.4276422 ])

In [ ]:
# Вычислим найденную перестановку
# (Сейчас совпало с аналитическим решением)
permutation_from_factoradic(
 qaoaInstance_4_1.calc_factoradic(optimizeResult_4_1.x)   
)

array([1, 0, 2, 3])

### Повторим процедуру для другого числа слоёв

In [ ]:
qaoaInstance_4_2 = FactoradicCirquit(4, 2)
qaoaInstance_4_2

0: ───Rx(a_0)───@───────────────────Rx(a_6)────@───────────────────
                │                              │
1: ───Rx(a_1)───X───@───────────────Rx(a_7)────X───@───────────────
                    │                              │
2: ───Rx(a_2)───────X───@───────────Rx(a_8)────────X───@───────────
                        │                              │
3: ───Rx(a_3)───────────X───@───────Rx(a_9)────────────X───@───────
                            │                              │
4: ───Rx(a_4)───────────────X───@───Rx(a_10)───────────────X───@───
                                │                              │
5: ───Rx(a_5)───────────────────X───Rx(a_11)───────────────────X───

In [ ]:
optimizeResult_4_2 = qaoaInstance_4_2.optimize_without_constraints(cost_matrix_4)
optimizeResult_4_2

                        fun: 0.52832923
 lowest_optimization_result:       fun: 0.52832923
 hess_inv: array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]])
      jac: array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
  message: 'Optimization terminated successfully.'
     nfev: 13
      nit: 0
     njev: 1
   status: 0
  success: True
        x: array([ 1.87255714, -3.67280128,  0.36844108, -2.04150122,  2.61531993,
        1.2071472 ,  1.26572708,  0.44710062, -2.34341103,  2.75215245,
        2.19068

In [ ]:
# Найдённое решение совпало с аналитическим
permutation_from_factoradic(
 qaoaInstance_4_2.calc_factoradic(optimizeResult_4_2.x)   
)

array([1, 0, 2, 3])

### Повторим процедуру c бóльшим числом городов

In [ ]:
# n_towns = 6

In [ ]:
# cost_matrix_6 = create_cost_matrix(6)

In [ ]:
cost_matrix_6 = np.array([[0.        , 0.641366  , 0.91674456, 0.6834148 , 0.72798224, 0.70088125],
                          [0.641366  , 0.        , 0.52118999, 0.97268979, 0.04808934, 0.55098949],
                          [0.91674456, 0.52118999, 0.        , 0.2881217 , 0.94889384, 0.41340382],
                          [0.6834148 , 0.97268979, 0.2881217 , 0.        , 0.67249453, 0.75849091],
                          [0.72798224, 0.04808934, 0.94889384, 0.67249453, 0.        , 0.81587317],
                          [0.70088125, 0.55098949, 0.41340382, 0.75849091, 0.81587317, 0.        ]])

In [ ]:
analytical(cost_matrix_6)

{'min_cost': 1.98401915, 'order_of_towns': array([4, 1, 5, 2, 3, 0])}

In [ ]:
qaoaInstance_6_1 = FactoradicCirquit(6, 1)
qaoaInstance_6_1

0: ────Rx(a_0)────@───────────────────────────────────────────────────────
                  │
1: ────Rx(a_1)────X───@───────────────────────────────────────────────────
                      │
2: ────Rx(a_2)────────X───@───────────────────────────────────────────────
                          │
3: ────Rx(a_3)────────────X───@───────────────────────────────────────────
                              │
4: ────Rx(a_4)────────────────X───@───────────────────────────────────────
                                  │
5: ────Rx(a_5)────────────────────X───@───────────────────────────────────
                                      │
6: ────Rx(a_6)────────────────────────X───@───────────────────────────────
                                          │
7: ────Rx(a_7)────────────────────────────X───@───────────────────────────
                                              │
8: ────Rx(a_8)────────────────────────────────X───@───────────────────────
                                                  │
9

In [ ]:
# 1 min; правильный результат получился с 3 попытки
optimizeResult_6_1 = qaoaInstance_6_1.optimize_without_constraints(cost_matrix_6)
optimizeResult_6_1

                        fun: 1.98401915
 lowest_optimization_result:       fun: 1.98401915
 hess_inv: array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]])
      jac: array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
  messag

In [ ]:
# Найденное решение совпало с аналитическим, но города идут в обратном порядке
permutation_from_factoradic(
 qaoaInstance_6_1.calc_factoradic(optimizeResult_6_1.x)   
)

array([4, 1, 5, 2, 3, 0])

### Повторим процедуру с другими классическими алгоритмами

In [ ]:
# 14 min
optimizeResult_6_1_dual_annealing = \
      qaoaInstance_6_1.optimize_without_constraints(cost_matrix_6, scipy.optimize.dual_annealing)

# Правильный результат fun с первой попытки
optimizeResult_6_1_dual_annealing

     fun: 1.98401915
 message: ['Maximum number of iteration reached']
    nfev: 30065
    nhev: 0
     nit: 1000
    njev: 4
  status: 0
 success: True
       x: array([-1.27603555, -0.12620801,  1.71307007,  1.88765894, -0.56034365,
       -1.10659781,  2.63195353,  1.39154285,  2.44487135, -1.73855537,
       -2.45242554, -2.54673846,  1.78854471, -0.88641502,  0.62889439])

In [ ]:
# 22 min
optimizeResult_6_1_differential_evolution = \
      qaoaInstance_6_1.optimize_without_constraints(cost_matrix_6, scipy.optimize.differential_evolution)

# Правильный результат fun с первой попытки
optimizeResult_6_1_differential_evolution

     fun: 1.98401915
 message: 'Optimization terminated successfully.'
    nfev: 45916
     nit: 203
 success: True
       x: array([-1.07983279,  1.23363206, -1.85081767, -1.75983306, -0.77134198,
        1.3020096 ,  2.51769799,  0.76962934,  1.97038225, -1.85314893,
        1.60192788,  2.44207473, -1.72526104,  0.56719756,  0.86617056])

### Когда алгоритм не справляется

In [ ]:
# Увеличим число слоёв до двух
qaoaInstance_6_2 = FactoradicCirquit(6, 2)
qaoaInstance_6_2

0: ────Rx(a_0)────@───────────────────────────────────────────────────────Rx(a_15)───@───────────────────────────────────────────────────────
                  │                                                                  │
1: ────Rx(a_1)────X───@───────────────────────────────────────────────────Rx(a_16)───X───@───────────────────────────────────────────────────
                      │                                                                  │
2: ────Rx(a_2)────────X───@───────────────────────────────────────────────Rx(a_17)───────X───@───────────────────────────────────────────────
                          │                                                                  │
3: ────Rx(a_3)────────────X───@───────────────────────────────────────────Rx(a_18)───────────X───@───────────────────────────────────────────
                              │                                                                  │
4: ────Rx(a_4)────────────────X───@─────────────────────────

In [ ]:
# 2 min
qaoaInstance_6_2.optimize_without_constraints(cost_matrix_6)

                        fun: 2.0285865899999997
 lowest_optimization_result:       fun: 2.0285865899999997
 hess_inv: array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 

Не совпало с костом аналитического решения (1.984). Я сделал 5 попыток.

Я попробовал использовать другие алгоритмы для оптимизации (`scipy.optimize.dual_annealing` и `differential_evolution`), но они работали дольше 40 минут и я их прервал.

По-видимому, два слоя -- это слишком много.

Семь городов требуют 21 кубит и считаются бесконечно долго.

Восемь городов требуют 28 кубитов. Cirq не может такое промоделировать из-за

(1) ошибок округления  

(2) и из-за того, что в Google Colab недостаточно оперативной памяти.

## Вывод

1. С ростом количества параметров классический оптимизатор в QAOA работает всё хуже и хуже. 


2. Кажется, что наличие квантовой схемы делает оптимизацию только сложнее.

 Можно было с тем же успехом вместо квантовой цепи использовать какую-то произвольную формулу с параметром, скажем, 
$$x_n = \operatorname{round}\bigl(\sin^2(\theta_n)\bigr), \qquad n = 1,\: 2 \ldots \text{n_qubits}$$
На основе битов $x_n$ можно вычислить перестановку и её стоимость. Параметры $\theta_n$ можно варьировать.

 Всё будет подобно коду, написанному выше. Только вместо квантовой цепи будет классическая формула.

 Я не проверял, но думаю, что тогда алгоритм оптимизации сойдётся за плюс-минус такое же количество итераций, что и в случае наличия квантовой цепи.

3. Возможно, стоит использовать какой-то другой набор гейтов вместо `Rx` и `CNOT`. 

4. Но, кажется, чтобы квантовая цепь давала хоть какой-то выигрыш, эволюция состояния должна зависеть от матрицы расстояний между городами. Цепь должна как бы знать кост-функцию.

 Я не знаю, какие есть хорошие способы этого добиться.